In [3]:
import json
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from mixpanel import Mixpanel

# Load credentials
project_root = os.path.dirname(os.getcwd())
dotenv_path = os.path.join(project_root, '.env.local')
load_dotenv(dotenv_path=dotenv_path)
SA_USERNAME = os.getenv("MIXPANEL_SERVICE_ACCOUNT_USERNAME").strip()
SA_SECRET = os.getenv("MIXPANEL_SERVICE_ACCOUNT_SECRET").strip()
PROJECT_ID = os.getenv("MIXPANEL_PROJECT_ID").strip()

# Define the API endpoint and parameters
API_ENDPOINT = 'https://data.mixpanel.com/api/2.0/export'
headers = {"accept": "text/plain"}
params = {
    'from_date': '2025-08-01',
    'to_date': '2025-08-07',
    'project_id': PROJECT_ID
}

# Make the authenticated API request
response = requests.get(
    API_ENDPOINT,
    headers=headers,
    params=params,
    auth=(SA_USERNAME, SA_SECRET)
)

# 4. Check for a successful response and parse the data
# The API returns data as newline-delimited JSON
if response.status_code == 200:
    raw_data = []
    # Split the response text by newline characters
    lines = response.text.strip().split('\n')
    for line in lines:
        try:
            # Each line is a separate JSON object (an event)
            raw_data.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Warning: Could not decode line: {line}")
    
    # Convert the list of events into a Pandas DataFrame
    df = pd.DataFrame(raw_data)
    print(f"Successfully fetched {len(df)} events.")
    print(df.head())
else:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.text)

Successfully fetched 23792 events.
                               event  \
0           ApplicantViewedHelpTopic   
1  ApplicantClickedCBVInvitationLink   
2                        CbvPageView   
3           ApplicantViewedAgreement   
4          ApplicantConsentedToTerms   

                                          properties  
0  {'time': 1754388692, 'distinct_id': '241211968...  
1  {'time': 1754446410, 'distinct_id': 'applicant...  
2  {'time': 1754446410, 'distinct_id': 'applicant...  
3  {'time': 1754446411, 'distinct_id': 'applicant...  
4  {'time': 1754446420, 'distinct_id': 'applicant...  


In [4]:
lines

['{"event":"ApplicantViewedHelpTopic","properties":{"time":1754388692,"distinct_id":"2412119686","$city":"Clifton","$insert_id":"CgihwflfmkCqaBbujeqxsjdEAgwAupzFxhrm","$lib_version":"2.3.0","$mp_api_endpoint":"api.mixpanel.com","$mp_api_timestamp_ms":1754388692810,"$region":"New Jersey","browser":"Chrome","cbv_applicant_id":null,"cbv_flow_id":null,"client_agency_id":null,"device_name":null,"device_type":"desktop","flow_started_seconds_ago":null,"invitation_id":null,"locale":"en","mp_country_code":"US","mp_lib":"ruby","mp_processing_time_ms":1754388692867,"timestamp":1754388692,"topic":"provider","user_agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"}}',
 '{"event":"ApplicantClickedCBVInvitationLink","properties":{"time":1754446410,"distinct_id":"applicant-117256","$city":"Phoenix","$insert_id":"olicrElvuiejqacfkBcxaFFhhBydqkFFaEED","$lib_version":"2.3.0","$mp_api_endpoint":"api.mixpanel.com","$mp_api_timestam